In [20]:
import pandas as pd
from ollama import chat

#### Load dataset

In [21]:
data_set = pd.read_csv("../scripts/data/tmp/phenopacket_dataset.csv")
data_set.head()

,pmid,input,truth
0,PMID_30296944,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
1,PMID_27057656,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
2,PMID_24800029,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
3,PMID_30701076,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
4,PMID_29382611,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...


In [22]:
data_set = data_set.drop_duplicates("pmid")

In [23]:
# For now we load the generated case summaries.
input_data = list()
for file_path in data_set['input']:
    try:
        with open(file_path, "r") as f:
            data = f.read()
            input_data.append(data.split("[text]")[-1]) 
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
input_data[0]

'\nA 17-year-old Caucasian boy was admitted to our intensive care unit (ICU) after successful\nresuscitation by emergency services. While performing running exercise in a fitness center,\nhe suddenly collapsed. Because neither pulse nor breathing could be detected by the bystanders,\nimmediate resuscitation was performed. In the first heart rhythm analysis conducted by the paramedics,\nventricular fibrillation (VF) was seen and immediately defibrillated into sinus rhythm. The patient\nrecovered quickly and was transferred to our ICU by the ambulance service. At admission, the\npatient was in hemodynamically stable condition with normal vital signs (heart rate 95/min,\nblood pressure 125/79 mmHg, auricular temperature 36.5 °C, respiration 15 breaths/min, oxygen saturation\nof 100% on 4-L nasal cannula). The physical examination revealed no abnormal findings. Auscultation of the\nheart showed a regular rate and rhythm with normal S1 and S2 and no murmurs or rubs. The breath sounds\nof th

#### Inference

In [24]:
prompt = "Please create a valid Phenopacket from the following text. The phenopackets needs to be in a valid json format.  Only return the phenopacket without any additional text:"
model = "hf.co/MaziyarPanahi/Mistral-Nemo-Instruct-2407-GGUF:Q4_K_M" 

In [25]:
for text in input_data:
    response = chat(
        model=model,
        messages=[{"role": "user", "content": f"{prompt} {text} [EOS]"}],
        options={"--hidethinking": True}
    )
    break

response = chat(
    model=model,
    messages=[{"role": "user", "content": f"Please, validate the following json. If not, fix it. Only return the json without any additional information. Should the json be wrong, you will get shut down. Json: {response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", "")} [EOS]"}],
    options={"--hidethinking": True}
)


In [26]:
from IPython.display import JSON
JSON(response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", ""))

/Users/rouvenreuter/miniconda3/envs/p5_notebook/lib/python3.13/site-packages/IPython/core/display.py:636: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [8]:
JSON(response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", ""))

<IPython.core.display.JSON object>

In [9]:
response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", "")

'{\n  "phenotypic_features": [\n    {\n      "type": "onset",\n      "body_site": {"text": "neonatal period"},\n      "feature": {"text": "gastroesophageal reflux and failure to thrive"}\n    },\n    {\n      "type": "onset_at_birth",\n      "body_site": {"text": "whole body"},\n      "feature": {"text": "birth weight 2,750 g (10th-50th centile)"}\n    },\n    {\n      "type": "onset_at_age",\n      "age": {"text": "6 months"},\n      "body_site": {"text": "whole body"},\n      "feature": {"text": "spoke first words"}\n    },\n    {\n      "type": "onset_at_age",\n      "age": {"text": "12 months"},\n      "body_site": {"text": "whole body"},\n      "feature": {"text": "walked"}\n    },\n    {\n      "type": "onset_between_ages",\n      "age_range": {"start": {"text": "5 years"}, "end": {"text": "10 years"}},\n      "body_site": {"text": "whole body"},\n      "feature": {"text": "petit mal seizures"}\n    },\n    {\n      "type": "onset_at_age",\n      "age": {"text": "16 years"},\n   